# 5. 处理任务中的异常

In [1]:
using System.Threading;
using System.Threading.Tasks;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
public static class TaskClass
{
    public static int TaskMethod(string taskname, int seconds)
    {
        Console.WriteLine("Task {taskname} is running on a thread id {Thread.CurrentThread.ManagedThreadId}. Is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(TimeSpan.FromSeconds(seconds));
        throw new Exception($"Task {taskname} Boom!");
        
        return 42 * seconds;
    }    
}


(9,9): warning CS0162: Unreachable code detected



## 5.1 单个任务

In [3]:
try
{
    Task<int> task = Task.Run(() => TaskClass.TaskMethod("Task 1", 2));
    int result = task.GetAwaiter().GetResult();
    Console.WriteLine("Result: {0}", result);
}
catch (Exception ex)
{
    Console.WriteLine("Task 1 Exception caught: {0}", ex.Message);
}

Console.WriteLine("done");

Task {taskname} is running on a thread id {Thread.CurrentThread.ManagedThreadId}. Is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}
Task 1 Exception caught: Task Task 1 Boom!
done


## 5.2 多个任务

In [4]:
try
{
    var t1 = new Task<int>(() => TaskClass.TaskMethod("Task 1", 3));
    var t2 = new Task<int>(() => TaskClass.TaskMethod("Task 2", 2));
    var complexTask = Task.WhenAll(t1, t2);  // t1 t2都完成
    var exceptionHandler = complexTask.ContinueWith(t =>
        Console.WriteLine("Result: {0}", t.Result),
        TaskContinuationOptions.OnlyOnFaulted
    );
    t1.Start();
    t2.Start();
    Task.WaitAll(t1, t2);
}
catch (AggregateException ex)
{
    ex.Handle(exception =>
    {
        Console.WriteLine(exception.Message);
        return true;
    });
}

Console.WriteLine("done");

Task {taskname} is running on a thread id {Thread.CurrentThread.ManagedThreadId}. Is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}
Task {taskname} is running on a thread id {Thread.CurrentThread.ManagedThreadId}. Is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}
Task Task 1 Boom!
Task Task 2 Boom!
done


## 5.3 async/await的方式

In [5]:
Task task = ObserveOneExceptionAsync();
Console.WriteLine("主线程继续运行... ...");
task.Wait();

public static async Task ObserveOneExceptionAsync()
{
    var task1 = ThrowNotImplementedExceptionAsync();
    var task2 = ThrowInvalidOperationExceptionAsync();
    var task3 = Normal();

    try
    {
        //异步的方式
        Task allTasks = Task.WhenAll(task1, task2, task3);
        await allTasks;

        //同步的方式
        //Task.WaitAll(task1, task2, task3);
    }
    catch (NotImplementedException ex)
    {
        Console.WriteLine("task1 任务报错!");
    }
    catch (InvalidOperationException ex)
    {
        Console.WriteLine("task2 任务报错!");
    }
    catch (Exception ex)
    {
        Console.WriteLine("任务报错!");
    }
}

public static Task Fun()
{
    return Task.Run(() =>
    {
        for (int i = 1; i <= 10; i++)
        {
            Console.WriteLine("i={0}", i);
            Thread.Sleep(200);
        }
    });
}

public static async Task Normal()
{
    await Fun();
}

public static async Task ThrowNotImplementedExceptionAsync()
{
    throw new NotImplementedException();
}

public static async Task ThrowInvalidOperationExceptionAsync()
{
    throw new InvalidOperationException();
}

i=1
主线程继续运行... ...
i=2
i=3
i=4
i=5
i=6
i=7
i=8
i=9
i=10
task1 任务报错!



(20,36): warning CS0168: The variable 'ex' is declared but never used

(24,38): warning CS0168: The variable 'ex' is declared but never used

(28,22): warning CS0168: The variable 'ex' is declared but never used

(51,26): warning CS1998: This async method lacks 'await' operators and will run synchronously. Consider using the 'await' operator to await non-blocking API calls, or 'await Task.Run(...)' to do CPU-bound work on a background thread.

(56,26): warning CS1998: This async method lacks 'await' operators and will run synchronously. Consider using the 'await' operator to await non-blocking API calls, or 'await Task.Run(...)' to do CPU-bound work on a background thread.

